In [1]:
!pip install onnx onnxruntime num2words

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=833664320548fd579d64b50904986c0722b66b56cb416e3542930fa41c9ebe39
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [2]:
! git clone https://github.com/huggingface/lerobot.git
! cd lerobot && pip install .

Cloning into 'lerobot'...
remote: Enumerating objects: 38249, done.
remote: Counting objects: 100% (1391/1391), done.
remote: Compressing objects: 100% (583/583), done.
remote: Total 38249 (delta 1107), reused 883 (delta 764), pack-reused 36858 (from 4)
Receiving objects: 100% (38249/38249), 169.36 MiB | 21.74 MiB/s, done.
Resolving deltas: 100% (24730/24730), done.
Filtering content: 100% (45/45), 69.04 MiB | 48.49 MiB/s, done.
Processing /content/lerobot
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
%pdb on

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.5 MB/s eta 0:00:00
env: PYTHONBREAKPOINT=IPython.core.debugger.set_trace  # makes breakpoint() use ipdb
Automatic pdb calling has been turned ON


In [4]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig

config = SmolVLAConfig
policy = SmolVLAPolicy.from_pretrained("lerobot/smolvla_base")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

processor_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Reducing the number of VLM layers to 16 ...


model.safetensors:   0%|          | 0.00/907M [00:00<?, ?B/s]

[standardise_state_dict] 'normalize_inputs.buffer_observation_state.mean'  ←  ['normalize_inputs.so100-red_buffer_observation_state.mean', 'normalize_inputs.so100_buffer_observation_state.mean']
[standardise_state_dict] 'normalize_inputs.buffer_observation_state.std'  ←  ['normalize_inputs.so100-red_buffer_observation_state.std', 'normalize_inputs.so100_buffer_observation_state.std']
[standardise_state_dict] 'normalize_targets.buffer_action.mean'  ←  ['normalize_targets.so100-red_buffer_action.mean', 'normalize_targets.so100_buffer_action.mean']
[standardise_state_dict] 'normalize_targets.buffer_action.std'  ←  ['normalize_targets.so100-red_buffer_action.std', 'normalize_targets.so100_buffer_action.std']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.mean'  ←  ['unnormalize_outputs.so100-red_buffer_action.mean', 'unnormalize_outputs.so100_buffer_action.mean']
[standardise_state_dict] 'unnormalize_outputs.buffer_action.std'  ←  ['unnormalize_outputs.so100-red_buffer_action.

In [ ]:
import torch
from torch import nn
from lerobot.policies.smolvla.smolvlm_with_expert import SmolVLMWithExpertModel

from typing import Optional, List

In [25]:
class SmolVLMVision(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, image: torch.Tensor):
        return self.model.embed_image(image)

In [26]:
class SmolVLMText(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

    def forward(self, tokens: torch.Tensor):
        return self.model.embed_language_tokens(tokens)

In [5]:
class SmolVLMForONNXExport(nn.Module):
    def __init__(self, model_id, **kwargs):
        super().__init__()
        self.model = policy.model.vlm_with_expert

        self.config = SmolVLAConfig

        # Bake in the configuration for the static graph
        self.attention_mode = self.config.vlm_model_name
        self.num_vlm_layers = self.model.num_vlm_layers
        self.num_expert_layers = self.model.num_expert_layers
        self.fill_kv_cache = False
        self.use_cache = True

    def embed_img(self,
                  image: torch.Tensor):
          return self.model.embed_image(image)

    def embed_text(self,
                   tokens: torch.Tensor):
        return self.model.embed_language_tokens(tokens)

    def forward(
          self,
          vlm_embeds: torch.Tensor,
          expert_embeds: torch.Tensor,
          attention_mask: torch.Tensor,
          position_ids: torch.LongTensor,
          # Flattened past_key_values: one tensor for keys, one for values per layer
          *past_key_values_flat: torch.Tensor):


          inputs_embeds = [vlm_embeds, expert_embeds]

          past_key_values = {}
          if past_key_values_flat:
              # Logic to reconstruct the dictionary from the flat tuple of tensors
              # This assumes a consistent ordering: (key_layer0, val_layer0, key_layer1, val_layer1, ...)
              for i in range(0, len(past_key_values_flat), 2):
                  layer_idx = i // 2
                  past_key_values[layer_idx] = {
                      "key_states": past_key_values_flat[i],
                      "value_states": past_key_values_flat[i+1],
                  }
          else:
              past_key_values = None


          final_embeds_list, new_past_key_values = self.model.forward(
              attention_mask=attention_mask,
              position_ids=position_ids,
              past_key_values=past_key_values,
              inputs_embeds=inputs_embeds,
              use_cache=self.use_cache,
              fill_kv_cache=self.fill_kv_cache,
          )

          # --- 3. Flatten Outputs ---
          # The output must be a flat tuple of tensors
          vlm_output_embeds = final_embeds_list[0]
          expert_output_embeds = final_embeds_list[1] # expects to be None

          present_key_values_flat = []
          if new_past_key_values:
            for i in range(self.num_vlm_layers):
                if i in new_past_key_values:
                    present_key_values_flat.append(new_past_key_values[i]["key_states"])
                    present_key_values_flat.append(new_past_key_values[i]["value_states"])

          return (vlm_output_embeds, expert_output_embeds, *present_key_values_flat)

In [6]:
import torch

onnx_model = SmolVLMForONNXExport("")

Prefil model condifuration

In [7]:
onnx_model.eval()

onnx_model.fill_kv_cache = True

onnx_model.to("cuda:0")

# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["vlm_output_embeds"]
num_layers = onnx_model.model.num_vlm_layers
for i in range(num_layers):
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size

batch_size = 1
vlm_seq_len = 256
expert_seq_len = 16
total_seq_len = vlm_seq_len + expert_seq_len

dummy_vlm_embeds = torch.randn(batch_size, vlm_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
# dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, 320, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_expert_embeds = torch.randn(batch_size, expert_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
# dummy_expert_embeds = torch.randn(batch_size, 320, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16) # input_embeds
dummy_attn_mask = torch.ones(batch_size, total_seq_len, total_seq_len, device="cuda:0", dtype=torch.bool)
dummy_pos_ids = torch.arange(total_seq_len, device="cuda:0").unsqueeze(0)


# dummy_inputs = (dummy_vlm_embeds, dummy_expert_embeds, dummy_attn_mask, dummy_pos_ids)
dummy_inputs = (dummy_vlm_embeds, None, dummy_attn_mask, dummy_pos_ids)

# we ant past_key_values from it, the second output



In [8]:
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_prefill.onnx",
    input_names=["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"],
    output_names=output_names,
    opset_version=17
)

/usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/smolvlm_with_expert.py:237: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len < position_ids.shape[1]:


In [9]:
_, _, *past_key_values = onnx_model.forward(*dummy_inputs)

In [10]:
past_key_values[0].shape

torch.Size([1, 256, 5, 64])

In [11]:
import torch


onnx_model = SmolVLMForONNXExport("")

Decode model configuration and export

In [12]:

onnx_model.to("cuda:0")
onnx_model.eval()
onnx_model.fill_kv_cache = False
onnx_model.use_cache = True
current_seq_len = 1
past_seq_len = 256

batch_size = 1

vlm_hidden_dim = onnx_model.model.config.text_config.hidden_size
expert_hidden_dim = onnx_model.model.expert_hidden_size
num_layers = onnx_model.model.num_vlm_layers
num_kv_heads = onnx_model.model.config.text_config.num_key_value_heads
head_dim = vlm_hidden_dim // onnx_model.model.config.text_config.num_attention_heads

dummy_vlm_embeds = torch.randn(batch_size, current_seq_len, 960, device="cuda:0", dtype=torch.bfloat16)
dummy_expert_embeds = torch.randn(batch_size, current_seq_len, expert_hidden_dim, device="cuda:0", dtype=torch.bfloat16)
# dummy_attn_mask = torch.ones(batch_size, current_seq_len, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
dummy_attn_mask = torch.ones(batch_size, current_seq_len, current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len, past_seq_len + current_seq_len + 1, device="cuda:0", dtype=torch.bool)
# dummy_attn_mask = torch.ones(batch_size, 1, past_seq_len + current_seq_len, device="cuda:0", dtype=torch.bool)
# dummy_pos_ids = torch.arange(past_seq_len + current_seq_len, device="cuda:0").unsqueeze(0)
dummy_pos_ids = torch.arange(current_seq_len, device="cuda:0").unsqueeze(0)

print(past_seq_len + current_seq_len)
print(total_seq_len)

input_names = ["vlm_embeds", "expert_embeds", "attention_mask", "position_ids"]

# past_key_values_flat = []
# for _ in range(num_layers):
#     key_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     val_shape = (batch_size, num_kv_heads, past_seq_len, head_dim)
#     past_key_values_flat.append(torch.randn(key_shape, device="cuda:0", dtype=torch.bfloat16))
#     past_key_values_flat.append(torch.randn(val_shape, device="cuda:0", dtype=torch.bfloat16))

dummy_inputs = (
    # dummy_vlm_embeds,
    None,
    dummy_expert_embeds,
    dummy_attn_mask,
    dummy_pos_ids,
    *past_key_values
    # None
)


for i in range(num_layers):
    input_names.append(f"past_key_{i}")
    input_names.append(f"past_value_{i}")


# Outputs include the embeddings and the *updated* KV cache
# output_names = ["vlm_output_embeds", "expert_output_embeds"]
output_names = ["expert_output_embeds"]
for i in range(num_layers):
    # The output names must be consistent for the next iteration
    output_names.append(f"present_key_{i}")
    output_names.append(f"present_value_{i}")

# dynamic_axes = {"vlm_embeds": {}, "expert_embeds": {}} # Add other static inputs if needed
# for i in range(num_layers):
#     # The '2' axis is the sequence length dimension in the KV cache shape
#     # (batch_size, num_heads, sequence_length, head_dim)
#     dynamic_axes[f"past_key_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"past_value_{i}"] = {2: "past_sequence_length"}
#     dynamic_axes[f"present_key_{i}"] = {2: "total_sequence_length"}
#     dynamic_axes[f"present_value_{i}"] = {2: "total_sequence_length"}

# we need output_embeds - first output
torch.onnx.export(
    onnx_model,
    dummy_inputs,
    "smolvlm_expert_decode.onnx",
    input_names=input_names,
    output_names=output_names,
    # dynamic_axes=dynamic_axes,
    opset_version=17,
    do_constant_folding=True,
)


257
272


In [24]:
onnx_model.model.get_vlm_model().vision_model

SmolVLMVisionTransformer(
  (embeddings): SmolVLMVisionEmbeddings(
    (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
    (position_embedding): Embedding(1024, 768)
  )
  (encoder): SmolVLMEncoder(
    (layers): ModuleList(
      (0-11): 12 x SmolVLMEncoderLayer(
        (self_attn): SmolVLMVisionAttention(
          (k_proj): Linear(in_features=768, out_features=768, bias=True)
          (v_proj): Linear(in_features=768, out_features=768, bias=True)
          (q_proj): Linear(in_features=768, out_features=768, bias=True)
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): SmolVLMVisionMLP(
          (activation_fn): PytorchGELUTanh()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        )
        (layer_norm2): LayerNorm((768,)

In [38]:
dummy_image = torch.ones(1, 3, 256, 256, device="cuda:0", dtype=torch.bfloat16) * 255
img_model = SmolVLMVision("")
img_model.to("cuda:0")
img_model.eval()

# torch.onnx.export()
emb = img_model.forward(dummy_image)
print(emb.shape)

torch.onnx.export(
    img_model,
    dummy_image,
    "smolvlm_vision.onnx",
    input_names=["image"],
    output_names=["output_embeds"],
    opset_version=17,
    do_constant_folding=False,
)

torch.Size([1, 16, 960])


/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:145: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for batch_idx, p_attn_mask in enumerate(patch_attention_mask):
/usr/local/lib/python3.12/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
/usr/local/lib/python3.12/dist-packages/transformers/models/smolvlm/modeling_smolvlm.py:532: TracerWarning: Converting a tensor to a Python integer might cause the 

In [ ]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_prefill.onnx"
out_dir = "splited"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )

In [ ]:
import os, onnx

out_name = None
src_name = "smolvlm_expert_decode.onnx"
out_dir = "splited_decoder"
os.makedirs(out_dir, exist_ok=True)
base = out_name or (os.path.splitext(os.path.basename(src_name))[0] + "_ext.onnx")
out_path = os.path.join(out_dir, base)

model = onnx.load(src_name)
onnx.save_model(
        model,
        out_path,
        save_as_external_data=True,
        all_tensors_to_one_file=True,
        location="weights.bin",
        size_threshold=0,
        convert_attribute=True
    )